# Make a single dataset

The goal of this notebook is to combine both datasets in a single one with matched beers.

## Breweries data

In [1]:
import os
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import copy
import ast

#dataset_path = Path(os.getcwd())  #Matteo
root = Path(os.getcwd()).parent.parent  #Val
dataset_path = os.path.join(root,'Dataset') #Val

ADVOCATE = "BeerAdvocate"
RATEBEER = "RateBeer"
MATCHED = "Matched"
FULL = "Full"


ADVOCATE = "ADVOCATE" #Val
RATEBEER = "RATEBEER" #Val
MATCHED = "MATCHED" #Val
FULL = "FULL" # Val

ADVOCATE_PATH = os.path.join(dataset_path,ADVOCATE)
RATEBEER_PATH = os.path.join(dataset_path,RATEBEER)
MATCHED_PATH = os.path.join(dataset_path,MATCHED)
FULL_PATH = os.path.join(dataset_path,FULL)

In [2]:
#Open this one already as it may become usefull for reweries already
beers_matched = pd.read_csv(os.path.join(MATCHED_PATH,'beers.csv'),header=1)

In [3]:
breweries_advocate = pd.read_csv(os.path.join(ADVOCATE_PATH,'breweries.csv'))
breweries_matched = pd.read_csv(os.path.join(MATCHED_PATH,'breweries.csv'),header=1)
breweries_ratebeer = pd.read_csv(os.path.join(RATEBEER_PATH,'breweries.csv'))

print("Length of the three datasets:\n-advocate:",len(breweries_advocate),"\n-matched:", len(breweries_matched),"\n-ratebeer:", len(breweries_ratebeer))

#Already delete the breweries presents in matched
breweries_ratebeer_solo = breweries_ratebeer[~breweries_ratebeer.id.isin(breweries_matched['id.1'])]
breweries_advocate_solo = breweries_advocate[~breweries_advocate.id.isin(breweries_matched['id'])]

Length of the three datasets:
-advocate: 16758 
-matched: 8281 
-ratebeer: 24189


In [4]:
breweries_duplicates_ratebeer = breweries_matched[breweries_matched['id.1'].duplicated(keep=False)]
breweries__not_duplicated_ratebeer = breweries_matched[~breweries_matched['id.1'].duplicated(keep=False)]#For later
breweries_duplicates_advocate = breweries_matched[breweries_matched['id'].duplicated(keep=False)]

print("Number of duplicated/tripled Ratbeer breweries",len(breweries_duplicates_ratebeer))
print("Number of duplicated/tripled Advocate breweries",len(breweries_duplicates_advocate))

Number of duplicated/tripled Ratbeer breweries 91
Number of duplicated/tripled Advocate breweries 0


In [8]:
#We are going to build a new dataset instead of playing with the dataset that contains every information for duplicates.

unique_ratebeer_brewery_id = breweries_duplicates_ratebeer['id.1'].unique()
new_matched_brewery_duplicate = pd.DataFrame(columns=['location', 'name', 'nbr_beers','old_advocate_id','old_ratebeer_id'])

for id in unique_ratebeer_brewery_id:
    # Define data for each row
    location = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['location.1'].iloc[0]#Always take the first as it is the same for the two/three of them
    name = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['name.1'].iloc[0]#Always take the first as it is the same for the two/three of them
    old_advocate_id = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['id']
    list_old_advocate_id = []
    for id_advocate in (old_advocate_id):
        list_old_advocate_id.append(id_advocate)
    old_ratebeer_id = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['id.1'].iloc[0]

    total_beers_advocate = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['nbr_beers'].sum()
    total_beers_ratebeer = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['nbr_beers.1'].iloc[0] #don't want to double the number of beers for the one duplicated

    nbr_beers = total_beers_ratebeer + total_beers_advocate

    # Append the new row to the DataFrame
    new_row = pd.DataFrame({'location': location, 'name': name, 'nbr_beers': nbr_beers,'old_advocate_id':[list_old_advocate_id],'old_ratebeer_id':old_ratebeer_id})
    new_matched_brewery_duplicate = pd.concat([new_matched_brewery_duplicate, new_row], ignore_index=True)

new_matched_brewery_duplicate.head(3)

,location,name,nbr_beers,old_advocate_id,old_ratebeer_id
0,England,Seven Bro7hers,11,"[37180, 45243]",20891
1,England,Dartmoor,19,"[25939, 22832]",3480
2,China,Great Leap Brewing,77,"[24935, 32111]",12325


In [9]:
new_matched_brewery_non_duplicate = copy.deepcopy(breweries__not_duplicated_ratebeer)
new_matched_brewery_non_duplicate['nbr_beers'] = new_matched_brewery_non_duplicate[['nbr_beers', 'nbr_beers.1']].sum(axis=1)
new_matched_brewery_non_duplicate = new_matched_brewery_non_duplicate.drop(columns=['location','name','diff', 'sim', 'nbr_beers.1'])
new_matched_brewery_non_duplicate = new_matched_brewery_non_duplicate.rename(columns={'location.1':'location','name.1':'name','id':'old_advocate_id', 'id.1':'old_ratebeer_id',})
new_matched_brewery_non_duplicate = new_matched_brewery_non_duplicate[['location', 'name', 'nbr_beers', 'old_advocate_id', 'old_ratebeer_id']]

display(new_matched_brewery_non_duplicate.head(3))

,location,name,nbr_beers,old_advocate_id,old_ratebeer_id
0,Northern Ireland,Strangford Lough,10,10093,4959
1,Northern Ireland,Sheelin,6,32848,17616
2,Northern Ireland,Walled City,9,40360,24866


In [10]:
#Concat the two datasets together
new_matched_brewery = pd.concat([new_matched_brewery_non_duplicate, new_matched_brewery_duplicate], ignore_index=True)
new_matched_brewery.sample(3)

,location,name,nbr_beers,old_advocate_id,old_ratebeer_id
8183,"United States, California",Rafters Grille & Brewery,1,13767,8573
5483,"United States, Mississippi",Lazy Magnolia Brewing Company,95,10990,5747
3078,Czech Republic,Purkmistr Pivovarský Dvůr Plzeň,52,18231,8740


### Check the previous split and concatenation operations

In [11]:
print(len(new_matched_brewery), len(new_matched_brewery_non_duplicate), len(new_matched_brewery_duplicate))
print(len(new_matched_brewery.old_ratebeer_id.unique()))

8235 8190 45
8235


### Number of beers

Now we need to count the number of beers that there really is. We assume that a beer can only be matched if its brewery is matched too.

In [12]:
new_matched_brewery_good_amount_rating = copy.deepcopy(new_matched_brewery)

# Count occurrences of each 'old_ratebeer_id' in 'beers_matched'
ratebeer_id_counts = beers_matched['brewery_id.1'].value_counts()

# Decrement 'nbr_beers' by the counts of each 'old_ratebeer_id' to remove the amount of beers that are already in the dataset
new_matched_brewery_good_amount_rating['nbr_beers'] -= new_matched_brewery_good_amount_rating['old_ratebeer_id'].map(ratebeer_id_counts).fillna(0).astype(int)

display(new_matched_brewery_good_amount_rating.head(3))

,location,name,nbr_beers,old_advocate_id,old_ratebeer_id
0,Northern Ireland,Strangford Lough,6,10093,4959
1,Northern Ireland,Sheelin,5,32848,17616
2,Northern Ireland,Walled City,8,40360,24866


In [13]:
breweries_ratebeer_solo = breweries_ratebeer_solo.rename(columns={'id': 'old_ratebeer_id'})
breweries_ratebeer_solo['old_advocate_id'] = np.nan
display(breweries_ratebeer_solo.sample(3))

breweries_advocate_solo = breweries_advocate_solo.rename(columns={'id': 'old_advocate_id'})
breweries_advocate_solo['old_ratebeer_id'] = np.nan
display(breweries_advocate_solo.sample(3))

full_breweries = pd.concat([new_matched_brewery_good_amount_rating,breweries_advocate_solo, breweries_ratebeer_solo], ignore_index=True)
full_breweries['id'] = range(1, len(full_breweries) + 1)
display(full_breweries.sample(10))

,old_ratebeer_id,location,name,nbr_beers,old_advocate_id
11082,18874,Netherlands,Brouwerij De Oude Rijn,3,NaN
3164,30616,Italy,Canapa Campana,1,NaN
20304,26595,England,Godstone Brewers,7,NaN


,old_advocate_id,location,name,nbr_beers,old_ratebeer_id
4300,6513,Germany,Schorschbräu,13,NaN
8989,21856,"United States, California",Strand Brewing Co.,26,NaN
4853,13271,Switzerland,Les Brasseurs,8,NaN


,location,name,nbr_beers,old_advocate_id,old_ratebeer_id,id
8459,England,Blue Bell Brewery Ltd,1,10194,NaN,8460
31776,France,Bam! Brasserie Artisanale Marsac,10,NaN,25284,31777
26911,"United States, California",Knotty Brewing,12,NaN,28672,26912
28663,"United States, New Mexico",Palmer Brewery and Cider House,4,NaN,26431,28664
27378,"United States, Massachusetts",John Harvards Brewhouse &#40;Corporate Office...,107,NaN,257,27379
10468,Germany,Rössle Bräu,1,18922,NaN,10469
25231,Netherlands,Keuvel Bier,5,NaN,26421,25232
3669,Norway,Nøisom Craft Beer,62,48218,16701,3670
9206,Canada,Les Brasseurs De Montebello,7,35272,NaN,9207
18964,Italy,Birrificio Serro Croce,3,NaN,18600,18965


In [17]:
display(full_breweries.head(2))
print(type(full_breweries['old_advocate_id'].iloc[0]))
print(full_breweries['old_advocate_id'].iloc[16712])


,location,name,nbr_beers,old_advocate_id,old_ratebeer_id,id
0,Northern Ireland,Strangford Lough,6,10093,4959,1
1,Northern Ireland,Sheelin,5,32848,17616,2


<class 'int'>
nan


In [18]:
print(len(full_breweries))
print(len(breweries_matched),len(breweries_advocate),len(breweries_ratebeer))
print(-len(breweries_matched)+len(breweries_advocate)+len(breweries_ratebeer))

32666
8281 16758 24189
32666


In [20]:
full_breweries.to_csv(os.path.join(FULL_PATH,'breweries.csv'),index=False)

test_breweri = pd.read_csv(os.path.join(FULL_PATH,'breweries.csv'))
print(len(test_breweri))
del test_breweri

32666


## Users data

We prefer to consider the users approx rather than the users of the matched dataset. We saw that the users do have duplicates (unlike the normal file). We decided to take the different users in the approx file as a single user even for a lower similitude value (no sim value is below 0.8006407690254358). We consider the effect of this to be negligeable as it is a small percent of data. (like in the brewery file).

In [2]:
users_matched = pd.read_csv(os.path.join(MATCHED_PATH,'users_approx.csv'),header=1)
users_advocate = pd.read_csv(os.path.join(ADVOCATE_PATH,'users.csv'))
users_ratebeer = pd.read_csv(os.path.join(RATEBEER_PATH,'users.csv'))

ratings_matched = pd.read_csv(os.path.join(MATCHED_PATH,'ratings.csv'),header=1)#needed 


print("Length of the three datasets:\n-advocate:",len(users_advocate),"\n-matched:", len(users_matched),"\n-ratebeer:", len(users_ratebeer))

#Already delete the breweries presents in matched
users_ratebeer_solo = users_ratebeer[~users_ratebeer.user_name.isin(users_matched['user_name.1'])]
users_advocate_solo = users_advocate[~users_advocate.user_name.isin(users_matched['user_name'])]

print("New length of:\n-advocate:",len(users_advocate_solo),"\n-ratebeer:", len(users_ratebeer_solo))

display(users_matched.head(2))

Length of the three datasets:
-advocate: 153704 
-matched: 3341 
-ratebeer: 70174
New length of:
-advocate: 150388 
-ratebeer: 66833


,joined,location,nbr_ratings,nbr_reviews,user_id,user_name,user_name_lower,joined.1,location.1,nbr_ratings.1,user_id.1,user_name.1,user_name_lower.1,sim
0,1.483009e+09,Spain,3,0,magicuenca.1185749,MAGICuenca,magicuenca,1.484046e+09,Spain,89,442761,MAGICuenca91,magicuenca91,0.904534
1,1.220868e+09,Germany,6,6,erzengel.248045,Erzengel,erzengel,1.224324e+09,Germany,8781,83106,Erzengel,erzengel,1.000000


In [3]:
users_duplicates_ratebeer = users_matched[users_matched['user_name.1'].duplicated(keep=False)]
users_duplicates_advocate = users_matched[users_matched['user_name'].duplicated(keep=False)]
users__not_duplicated_advocate = users_matched[~users_matched['user_name'].duplicated(keep=False)]#For later

print("Number of duplicated/tripled Ratbeer users",len(users_duplicates_ratebeer))
print("Number of duplicated/tripled Advocate users",len(users_duplicates_advocate))

Number of duplicated/tripled Ratbeer users 0
Number of duplicated/tripled Advocate users 47


Both advocate and ratebeer dataset have as columns ['user_id', 'location', 'user_name','user_name_lower', 'joined','nbr_ratings']. Advocate has nbr_reviews in addition. Matched has those columns with .1 referring to ratebeer. It also has a column 'sim' that we will drop. The end format we want to have: [general_id, old_user_id_advocate, old_user__id_ratebeer, location, user_name_lower, joined_advocate, joined_ratebeer and nbr_ratings]. We think that the other columns will not be usefull for our analysis. The next cell shows that the user_name_lower are equal between the two datasets. For the location we will in opposition to the brewery give priority to the advocate dataset, as it is this time the one having one correspondence to multiple ratebeer users.

In [4]:
display(users_duplicates_advocate.sample(3))
users_duplicates_advocate[users_duplicates_advocate['user_id']=='lonestar.677281'].head(2)

,joined,location,nbr_ratings,nbr_reviews,user_id,user_name,user_name_lower,joined.1,location.1,nbr_ratings.1,user_id.1,user_name.1,user_name_lower.1,sim
821,1.156068e+09,Canada,207,207,nighthawk.93517,nighthawk,nighthawk,1.299150e+09,Canada,4,124623,nighthawk55,nighthawk55,0.894427
1989,1.434622e+09,"United States, Idaho",1,0,ralphie9.1001278,Ralphie9,ralphie9,1.427540e+09,"United States, Idaho",1,363652,Ralphie9,ralphie9,1.000000
1589,1.365502e+09,Brazil,2,2,marcelo.727734,Marcelo,marcelo,1.146823e+09,Brazil,2,36866,Marcelo,marcelo,1.000000


,joined,location,nbr_ratings,nbr_reviews,user_id,user_name,user_name_lower,joined.1,location.1,nbr_ratings.1,user_id.1,user_name.1,user_name_lower.1,sim
562,1.337422e+09,"United States, Texas",1,1,lonestar.677281,Lonestar,lonestar,1.081332e+09,"United States, Texas",13,11446,oneStar,onestar,0.822609
2059,1.337422e+09,"United States, Texas",1,1,lonestar.677281,Lonestar,lonestar,1.162984e+09,"United States, Texas",6,44744,LONESTAR,lonestar,1.000000


In [5]:
#We are going to build a new dataset instead of playing with the dataset that contains every information for duplicates.

unique_advocate_user_user_name = users_duplicates_advocate['user_name'].unique()
new_matched_user_duplicate = pd.DataFrame(columns=['location', 'joined_advocate', 'old_user_id_advocate','joined_ratebeer','old_user_id_ratebeer','user_name','nbr_ratings_total'])

for user_name in unique_advocate_user_user_name:  # Replace with your actual loop condition
    # Define data for each row
    location = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['location'].iloc[0]#Always take the first as it is the same for the two/three of them
    joined_advocate = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['joined'].iloc[0]
    joined_ratebeer = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['joined.1'].min() #take min value of joined time (we want the first time they joined)

    old_user_id_advocate = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['user_id'].iloc[0]
    old_user_id_ratebeer = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['user_id.1']
    name = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['user_name'].iloc[0]

    list_old_ratebeer_id = []
    for id_ratbeer in (old_user_id_ratebeer):
        list_old_ratebeer_id.append(id_ratbeer)

    total_ratings_advocate = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['nbr_ratings'].iloc[0]
    total_ratings_ratebeer = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['nbr_ratings.1'].sum()

    nbr_ratings_total = total_ratings_advocate + total_ratings_ratebeer

    # Append the new row to the DataFrame
    new_row = pd.DataFrame({'location': location, 'user_name': name,'old_user_id_ratebeer':[list_old_ratebeer_id],'old_user_id_advocate':old_user_id_advocate,'joined_advocate':joined_advocate,'joined_ratebeer':joined_ratebeer,'nbr_ratings_total':nbr_ratings_total})
    new_matched_user_duplicate = pd.concat([new_matched_user_duplicate, new_row], ignore_index=True)

new_matched_user_duplicate.sample(10)


C:\Users\valen\AppData\Local\Temp\ipykernel_11984\2846442710.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_matched_user_duplicate = pd.concat([new_matched_user_duplicate, new_row], ignore_index=True)


,location,joined_advocate,old_user_id_advocate,joined_ratebeer,old_user_id_ratebeer,user_name,nbr_ratings_total
4,"United States, Texas",1.337422e+09,lonestar.677281,1.081332e+09,"[11446, 44744]",Lonestar,20
5,"United States, California",1.295521e+09,chadski.555343,1.381054e+09,"[282710, 397860]",chadski,406
14,Canada,1.121422e+09,beers.29246,1.185444e+09,"[58154, 241070, 130784]",Beers,13
8,"United States, Massachusetts",1.336298e+09,maximusmaximus.675527,1.108897e+09,"[20099, 356497]",Maximusmaximus,112
2,"United States, Florida",1.350209e+09,porterporter.698876,1.023790e+09,"[4759, 72217]",PorterPorter,7382
20,"United States, California",1.089886e+09,brewmaster.7302,1.309169e+09,"[130842, 246380]",BrewMaster,448
11,"United States, Indiana",1.399975e+09,indianabeerman.799605,1.070104e+09,"[9581, 283897]",indianabeerman,668
12,Netherlands,1.218103e+09,schuim.240905,1.007118e+09,"[2462, 105981]",schuim,79
21,"United States, Ohio",1.104318e+09,beerbeerbeerbeer.12454,1.107256e+09,"[19505, 411715]",beerbeerbeerbeer,8
19,"United States, Idaho",1.434622e+09,ralphie9.1001278,1.160388e+09,"[363652, 43370]",Ralphie9,12


In [6]:
users_matched_not_duplicated = users__not_duplicated_advocate.drop(columns=['nbr_reviews','user_name.1','user_name_lower','location','user_name_lower.1','sim'])
users_matched_not_duplicated = users_matched_not_duplicated.rename(columns={'joined': 'joined_advocate','joined.1': 'joined_ratebeer','user_id': 'old_user_id_advocate','user_id.1': 'old_user_id_ratebeer','location.1':'location'})
users_matched_not_duplicated["nbr_ratings_total"] = users_matched_not_duplicated.nbr_ratings+users_matched_not_duplicated["nbr_ratings.1"]
users_matched_not_duplicated = users_matched_not_duplicated.drop(columns=['nbr_ratings','nbr_ratings.1'])
users_matched_not_duplicated.sample(5)

,joined_advocate,old_user_id_advocate,user_name,joined_ratebeer,location,old_user_id_ratebeer,nbr_ratings_total
3301,1.230462e+09,cmilmerstadt.281546,cmilmerstadt,1.273918e+09,"United States, New York",106370,3
2579,1.489748e+09,tarynotto.1194367,tarynotto,1.430734e+09,"United States, Florida",368416,5
272,1.012043e+09,maxpower.235,maxpower,1.010920e+09,"United States, Missouri",2913,1588
157,1.294484e+09,awmiller.549772,awmiller,1.303121e+09,"United States, Texas",127140,448
989,1.289905e+09,steeeve.526236,Steeeve,1.271585e+09,"United States, Pennsylvania",104914,225


In [7]:
#Concat the two datasets together
new_matched_user = pd.concat([users_matched_not_duplicated, new_matched_user_duplicate], ignore_index=True)
new_matched_user.sample(3)

,joined_advocate,old_user_id_advocate,user_name,joined_ratebeer,location,old_user_id_ratebeer,nbr_ratings_total
2739,1.354964e+09,jnance34.706749,jnance34,1.376820e+09,"United States, North Carolina",275319,44
1551,1.086343e+09,sjjn.6634,sjjn,1.089367e+09,"United States, New York",13561,128
2240,1.243246e+09,xnicknj.333746,xnicknj,1.281521e+09,"United States, Pennsylvania",111487,2194


In [8]:
users_matched_not_duplicated_good_amount_ratings = copy.deepcopy(new_matched_user)

# Count occurrences of each 'old_ratebeer_id' in 'beers_matched'
ratings_matched_id_counts = ratings_matched['user_id'].value_counts()

# Decrement 'nbr_beers' by the counts of each 'old_ratebeer_id' to remove the amount of beers that are already in the dataset
users_matched_not_duplicated_good_amount_ratings['nbr_ratings_total'] -= users_matched_not_duplicated_good_amount_ratings['old_user_id_advocate'].map(ratings_matched_id_counts).fillna(0).astype(int)

print(new_matched_user.nbr_ratings_total.sum())
print(users_matched_not_duplicated_good_amount_ratings.nbr_ratings_total.sum())

1608147
1586183


In [9]:
# Refactor the dataframe to prepare for the merge
users_ratebeer_solo = users_ratebeer_solo.rename(columns={'user_id': 'old_user_id_ratebeer','joined':'joined_ratebeer','nbr_ratings':'nbr_ratings_total'})
users_ratebeer_solo['old_user_id_advocate'] = np.nan
users_ratebeer_solo['joined_advocate'] = np.nan

users_advocate_solo = users_advocate_solo.rename(columns={'user_id': 'old_user_id_advocate','joined':'joined_advocate','nbr_ratings':'nbr_ratings_total'})
users_advocate_solo['old_user_id_ratebeer'] = np.nan
users_advocate_solo['joined_ratebeer'] = np.nan
users_advocate_solo = users_advocate_solo.drop(columns=['nbr_reviews'])

# Check the former operations
print("Sample of users only present in Ratebeer:")
display(users_ratebeer_solo.sample(3))

print("Sample of users only present in BeerAdvocate:")
display(users_advocate_solo.sample(3))


# Merge the three dataframes
full_users = pd.concat([users_matched_not_duplicated_good_amount_ratings,users_ratebeer_solo, users_advocate_solo], ignore_index=True)
full_users['id'] = range(1, len(full_users) + 1)

# Check the former operation
print("Sample of all users:")
display(full_users.sample(2))

print("Total number of users: ", len(full_users))
print("Number of users present in both datasets: ", len(users_matched))
print("Number of users present in BeerAdvocate: ",len(users_advocate))
print("Number of users present in RateBeer: ",len(users_ratebeer))
print("Sum of all the users (for chekcing purposes): ", len(users_matched_not_duplicated_good_amount_ratings)+len(users_advocate_solo)+len(users_ratebeer_solo))

Sample of users only present in Ratebeer:


,nbr_ratings_total,old_user_id_ratebeer,user_name,joined_ratebeer,location,old_user_id_advocate,joined_advocate
12706,6,351670,Grenelvec,1.420456e+09,"United States, Nebraska",NaN,NaN
34163,1,313984,woutertjong,1.397902e+09,NaN,NaN,NaN
31064,2,181116,Psycho,1.339841e+09,NaN,NaN,NaN


Sample of users only present in BeerAdvocate:


,nbr_ratings_total,old_user_id_advocate,user_name,joined_advocate,location,old_user_id_ratebeer,joined_ratebeer
44296,11,bier_uber_alles.672104,bier_uber_alles,1.334138e+09,NaN,NaN,NaN
58753,36,kcc.784640,kcc,1.393240e+09,"United States, Pennsylvania",NaN,NaN
29022,7,rdb51.794918,RDB51,1.397642e+09,"United States, California",NaN,NaN


Sample of all users:


,joined_advocate,old_user_id_advocate,user_name,joined_ratebeer,location,old_user_id_ratebeer,nbr_ratings_total,id
31601,NaN,NaN,Redbutcher,1.450350e+09,NaN,394504,9,31602
202531,1.402654e+09,trishal9.805703,TrishaL9,NaN,"United States, Maine",NaN,3,202532


Total number of users:  220537
Number of users present in both datasets:  3341
Number of users present in BeerAdvocate:  153704
Number of users present in RateBeer:  70174
Sum of all the users (for chekcing purposes):  220537


In [10]:
test = full_users[full_users['user_name'].duplicated(keep=False)]
test[test['user_name']=='Elwood']

,joined_advocate,old_user_id_advocate,user_name,joined_ratebeer,location,old_user_id_ratebeer,nbr_ratings_total,id
990,1.152871e+09,elwood.88673,Elwood,1.374314e+09,Canada,270235,6,991
4952,NaN,NaN,Elwood,1.235041e+09,"United States, Virginia",87609,2966,4953


Oh so in different dataset people can have the same user_name. We need to be carefull about this. If we treat the data we need to use id.

In [11]:
# Save the dataframe to a csv file
full_users.to_csv(os.path.join(FULL_PATH,'users.csv'),index=False)

# Check the former operation
test_user = pd.read_csv(os.path.join(FULL_PATH,'users.csv'))
assert len(test_user) == len(full_users), f"Expected {len(full_users)}, but got {len(test_user)}"
del test_user

C:\Users\valen\AppData\Local\Temp\ipykernel_11984\896566721.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  test_user = pd.read_csv(os.path.join(FULL_PATH,'users.csv'))


## Beer data

In this dataset we saw that there were no duplicates. We do not need to make the carefull analysis we made before. However we still need to look at the matched dataset. We also need to link the new brewery id, but we also need to count the number of ratings.

In [3]:
beers_matched = pd.read_csv(os.path.join(MATCHED_PATH,'beers.csv'),header=1)
beers_advocate = pd.read_csv(os.path.join(ADVOCATE_PATH,'beers.csv'))
beers_ratebeer = pd.read_csv(os.path.join(RATEBEER_PATH,'beers.csv'))

ratings_matched = pd.read_csv(os.path.join(MATCHED_PATH,'ratings.csv'),header=1)#needed 
full_breweries_for_beers = pd.read_csv(os.path.join(FULL_PATH,'breweries.csv'))#needed


print("Length of the three datasets:\n-advocate:",len(beers_advocate),"\n-matched:", len(beers_matched),"\n-ratebeer:", len(beers_ratebeer))

#Already delete the breweries presents in matched
beers_ratebeer_solo = beers_ratebeer[~beers_ratebeer.beer_id.isin(beers_matched['beer_id.1'])]
beers_advocate_solo = beers_advocate[~beers_advocate.beer_id.isin(beers_matched['beer_id'])]

print("New length of:\n-advocate:",len(beers_advocate_solo),"\n-ratebeer:", len(beers_ratebeer_solo))

Length of the three datasets:
-advocate: 280823 
-matched: 45640 
-ratebeer: 442081
New length of:
-advocate: 235183 
-ratebeer: 396441


In [4]:
display(beers_matched.sample(2))
beers_new = beers_matched.drop(columns=['avg','beer_wout_brewery_name.1','avg.1','avg_matched_valid_ratings.1','nbr_reviews','beer_name.1','brewery_name.1','brewery_name','avg_computed','avg_computed.1','avg_matched_valid_ratings','ba_score','beer_wout_brewery_name','sim','diff','zscore','zscore.1','overall_score','style_score','nbr_matched_valid_ratings','nbr_matched_valid_ratings.1'])
display(beers_new.sample(2))
print((beers_new['abv'] == beers_new['abv.1']).all())

,abv,avg,avg_computed,avg_matched_valid_ratings,ba_score,beer_id,beer_name,beer_wout_brewery_name,brewery_id,brewery_name,...,brewery_id.1,brewery_name.1,nbr_matched_valid_ratings.1,nbr_ratings.1,overall_score,style.1,style_score,zscore.1,diff,sim
14347,4.5,3.59,3.439932,3.593488,82.0,65010,Perfect Tin Amber,Perfect Amber,24488,Tin Roof Brewing Company,...,11992,Tin Roof Brewing Company,61,61,44.0,Amber Ale,56.0,-0.482239,1.000000,1.0
42235,6.8,4.16,4.066167,4.177333,89.0,120922,Eleanor,Eleanor,34507,Pollyanna Brewing Company,...,19549,Pollyanna Brewing Company,17,17,74.0,Porter,61.0,0.032700,0.525661,1.0


,abv,beer_id,beer_name,brewery_id,bros_score,nbr_ratings,style,abv.1,beer_id.1,brewery_id.1,nbr_ratings.1,style.1
16994,4.5,230028,Calling Elvis,42562,NaN,1,Milk / Sweet Stout,4.5,444229,28029,1,Sweet Stout
4754,4.7,226244,Table Saison,35805,NaN,5,Saison / Farmhouse Ale,4.7,401479,19968,9,Saison


True


The alcohol by volume is exactly the same between the two datasets.

In [5]:
#We are going to build a new dataset instead of playing with the dataset that contains every information for duplicates.
#Not sure about valid ratings so prefer to keep it as it is for now
new_matched_beer = pd.DataFrame(columns=['abv', 'old_beer_id_advocate', 'old_beer_id_ratebeer','beer_name','brewery_id','bros_score','nbr_ratings','style_advocate','style_ratebeer'])

# Use a list to collect rows and create the DataFrame at once
rows = []

for index, row in beers_new.iterrows():
    # Define data for each row
    abv = row['abv']
    beer_id_advocate = row['beer_id']
    beer_id_ratebeer = row['beer_id.1']
    beer_name = row['beer_name']
    bros_score = row['bros_score']
    style_advocate = row['style']
    style_ratebeer = row['style.1']

    brewery_id = full_breweries_for_beers[full_breweries_for_beers['old_ratebeer_id']==row['brewery_id.1']]['id'].values[0]

    total_ratings_advocate = row['nbr_ratings']
    total_ratings_ratebeer = row['nbr_ratings.1']

    nbr_ratings_total = total_ratings_advocate + total_ratings_ratebeer

    # Append the new row to the list
    rows.append({'abv': abv, 'old_beer_id_advocate': beer_id_advocate, 'old_beer_id_ratebeer': beer_id_ratebeer, 'beer_name': beer_name, 'brewery_id': brewery_id, 'bros_score': bros_score, 'nbr_ratings': nbr_ratings_total, 'style_advocate': style_advocate, 'style_ratebeer': style_ratebeer})

# Create the DataFrame from the list of rows
new_matched_beer = pd.DataFrame(rows)

new_matched_beer.head(10)

,abv,old_beer_id_advocate,old_beer_id_ratebeer,beer_name,brewery_id,bros_score,nbr_ratings,style_advocate,style_ratebeer
0,4.8,19827,37923,Legbiter,1,80.0,164,English Pale Ale,Golden Ale/Blond Ale
1,6.0,20841,41286,St. Patrick's Ale,1,NaN,19,English Pale Ale,Irish Ale
2,4.2,20842,41287,St. Patrick's Best,1,90.0,138,English Bitter,Bitter
3,4.8,22659,41285,St. Patrick's Gold,1,NaN,5,American Pale Wheat Ale,Amber Ale
4,4.5,178681,230283,Sheelin Stout,2,NaN,2,Irish Dry Stout,Mild Ale
5,4.2,178689,368966,Boom,3,NaN,3,American Pale Ale (APA),American Pale Ale
6,4.6,169948,155699,Bally Black Stout,4,NaN,6,Irish Dry Stout,Stout
7,5.2,169950,160664,Pig Island Pale Ale,4,NaN,4,English Pale Ale,Bitter
8,4.4,169949,177517,Rockin’ Goose,4,NaN,3,English Pale Mild Ale,Irish Ale
9,4.9,169951,299091,Scrabo Gold,4,NaN,2,English Pale Ale,Golden Ale/Blond Ale


In [6]:
new_matched_beer_good_amount_ratings = copy.deepcopy(new_matched_beer)

# Count occurrences of each 'old_ratebeer_id' in 'beers_matched'
ratings_matched_beer_id_counts = ratings_matched['beer_id'].value_counts()

# Decrement 'nbr_beers' by the counts of each 'old_ratebeer_id' to remove the amount of beers that are already in the dataset
new_matched_beer_good_amount_ratings['nbr_ratings'] -= new_matched_beer_good_amount_ratings['old_beer_id_advocate'].map(ratings_matched_beer_id_counts).fillna(0).astype(int)

print(new_matched_beer.nbr_ratings.sum())
print(new_matched_beer_good_amount_ratings.nbr_ratings.sum())

1976606
1954642


In [7]:
beers_ratebeer_solo_new = beers_ratebeer_solo.copy()
beers_ratebeer_solo_new = beers_ratebeer_solo_new.drop(columns=['brewery_name','overall_score','style_score','avg','avg_computed','zscore','nbr_matched_valid_ratings','avg_matched_valid_ratings'])
beers_ratebeer_solo_new = beers_ratebeer_solo_new.rename(columns={'beer_id': 'old_beer_id_ratebeer','style':'style_ratebeer'})

beers_ratebeer_solo_new = pd.merge(beers_ratebeer_solo_new,full_breweries_for_beers[['old_ratebeer_id', 'id']],how='left', left_on='brewery_id',right_on='old_ratebeer_id')
beers_ratebeer_solo_new = beers_ratebeer_solo_new.drop(columns=['brewery_id','old_ratebeer_id'])
beers_ratebeer_solo_new = beers_ratebeer_solo_new.rename(columns={'id': 'brewery_id'})

beers_ratebeer_solo_new['old_beer_id_advocate'] = np.nan
beers_ratebeer_solo_new['bros_score'] = np.nan
beers_ratebeer_solo_new['style_advocate']=np.nan

beers_ratebeer_solo_new.sample(3)

,old_beer_id_ratebeer,beer_name,style_ratebeer,nbr_ratings,abv,brewery_id,old_beer_id_advocate,bros_score,style_advocate
214798,124671,Ruddy Duck Scottish Ale,Scottish Ale,1,5.0,5068,NaN,NaN,NaN
481,80726,Pump Room Lager,Pale Lager,17,4.5,595,NaN,NaN,NaN
132023,11379,Porterhouse Oyster Stout,Stout,633,5.2,26305,NaN,NaN,NaN


In [9]:
display(beers_advocate_solo.sample(1))
beers_advocate_solo_new = copy.deepcopy(beers_advocate_solo)
beers_advocate_solo_new = beers_advocate_solo_new.drop(columns=['brewery_name','nbr_reviews','ba_score','avg','avg_computed','zscore','nbr_matched_valid_ratings','avg_matched_valid_ratings'])
beers_advocate_solo_new = beers_advocate_solo_new.rename(columns={'beer_id': 'old_beer_id_advocate','style':'style_advocate'})

display(full_breweries_for_beers[full_breweries_for_beers['old_ratebeer_id']==20891].head(2))

result = [ast.literal_eval(x) if isinstance(x, str) else x for x in full_breweries_for_beers['old_advocate_id'].tolist()]
full_breweries_for_beers['old_advocate_id'] = pd.Series(result, index=full_breweries_for_beers.index)
full_breweries_for_beers = full_breweries_for_beers.explode('old_advocate_id')

beers_advocate_solo_new = pd.merge(beers_advocate_solo_new,full_breweries_for_beers[['old_advocate_id', 'id']],how='left', left_on='brewery_id',right_on='old_advocate_id')
beers_advocate_solo_new = beers_advocate_solo_new.drop(columns=['brewery_id','old_advocate_id'])
beers_advocate_solo_new = beers_advocate_solo_new.rename(columns={'id': 'brewery_id'})

beers_advocate_solo_new['old_beer_id_ratebeer'] = np.nan
beers_advocate_solo_new['style_ratebeer']=np.nan

display(beers_advocate_solo_new.head())

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
90809,124590,Hopped Up Monk,29200,Tahoe Mountain Brewing Co.,Belgian Strong Dark Ale,1,0,3.5,NaN,NaN,10.0,3.5,NaN,0,NaN


,location,name,nbr_beers,old_advocate_id,old_ratebeer_id,id
8190,England,Seven Bro7hers,10,"[37180, 45243]",20891.0,8191


,old_beer_id_advocate,beer_name,style_advocate,nbr_ratings,bros_score,abv,brewery_id,old_beer_id_ratebeer,style_ratebeer
0,166064,Nashe Moskovskoe,Euro Pale Lager,0,NaN,4.7,8236,NaN,NaN
1,166065,Nashe Pivovskoe,Euro Pale Lager,0,NaN,3.8,8236,NaN,NaN
2,166066,Nashe Shakhterskoe,Euro Pale Lager,0,NaN,4.8,8236,NaN,NaN
3,166067,Nashe Zhigulevskoe,Euro Pale Lager,0,NaN,4.0,8236,NaN,NaN
4,166063,Zhivoe,Euro Pale Lager,0,NaN,4.5,8236,NaN,NaN


In [10]:
full_beers = pd.concat([new_matched_beer_good_amount_ratings,beers_ratebeer_solo_new, beers_advocate_solo_new], ignore_index=True)
full_beers['id'] = range(1, len(full_beers) + 1)
full_beers.sample(2)

,abv,old_beer_id_advocate,old_beer_id_ratebeer,beer_name,brewery_id,bros_score,nbr_ratings,style_advocate,style_ratebeer,id
658938,6.0,210287.0,NaN,Courthouse Cookie Ale,7212,NaN,0,American Brown Ale,NaN,658939
114767,NaN,NaN,119727.0,Wichtel Sommerbier,7779,NaN,1,NaN,Pilsener,114768


In [11]:
print(len(full_beers))
print(len(beers_matched),len(beers_advocate),len(beers_ratebeer))
print(len(new_matched_beer_good_amount_ratings)+len(beers_advocate_solo_new)+len(beers_ratebeer_solo_new))

677264
45640 280823 442081
677264


In [12]:
full_beers.to_csv(os.path.join(FULL_PATH,'beers.csv'),index=False)

In [13]:
test_beer = pd.read_csv(os.path.join(FULL_PATH,'beers.csv'))
assert len(test_beer) == len(full_beers), f"Expected {len(full_beers)}, but got {len(test_beer)}"
del test_beer

C:\Users\valen\AppData\Local\Temp\ipykernel_21896\4153181371.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  test_beer = pd.read_csv(os.path.join(FULL_PATH,'beers.csv'))


## Ratings

In [1]:
import os
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import copy
import ast

#dataset_path = Path(os.getcwd())  #Matteo
root = Path(os.getcwd()).parent.parent  #Val
dataset_path = os.path.join(root,'Dataset') #Val

ADVOCATE = "BeerAdvocate"
RATEBEER = "RateBeer"
MATCHED = "Matched"
FULL = "Full"


ADVOCATE = "ADVOCATE" #Val
RATEBEER = "RATEBEER" #Val
MATCHED = "MATCHED" #Val
FULL = "FULL" # Val

ADVOCATE_PATH = os.path.join(dataset_path,ADVOCATE)
RATEBEER_PATH = os.path.join(dataset_path,RATEBEER)
MATCHED_PATH = os.path.join(dataset_path,MATCHED)
FULL_PATH = os.path.join(dataset_path,FULL)

In [2]:
ratings_matched = pd.read_csv(os.path.join(MATCHED_PATH,'ratings.csv'),header=1)
ratings_advocate = pd.read_csv(os.path.join(ADVOCATE_PATH,'ratings-advocate.csv'))
ratings_ratebeer = pd.read_csv(os.path.join(RATEBEER_PATH,'ratings.csv'))

print("Length of the three datasets:\n-advocate:",len(ratings_advocate),"\n-matched:", len(ratings_matched),"\n-ratebeer:", len(ratings_ratebeer))

Length of the three datasets:
-advocate: 8393032 
-matched: 21964 
-ratebeer: 7122074


In [3]:
full_users = pd.read_csv(os.path.join(FULL_PATH,'users.csv'))
full_beers = pd.read_csv(os.path.join(FULL_PATH,'beers.csv'))
full_breweries = pd.read_csv(os.path.join(FULL_PATH,'breweries.csv'))

C:\Users\valen\AppData\Local\Temp\ipykernel_9520\2158992435.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  full_users = pd.read_csv(os.path.join(FULL_PATH,'users.csv'))
C:\Users\valen\AppData\Local\Temp\ipykernel_9520\2158992435.py:2: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  full_beers = pd.read_csv(os.path.join(FULL_PATH,'beers.csv'))


In [4]:
ratings_advocate = ratings_advocate.drop(columns=['review','beer_name','brewery_name','style','user_name','abv'])
ratings_matched = ratings_matched.drop(columns=['review','beer_name','beer_name.1','brewery_name','brewery_name.1','style','style.1','user_name','user_name.1','abv','abv.1'])
ratings_ratebeer = ratings_ratebeer.drop(columns=['beer_name','brewery_name','style','user_name','abv'])

In contradiction to the previous method, we would like to keep both comments and grades given the the datasets for a macthed comments. We want to do this as the text might differ and the grades too. First we just want to add a column 'dataset' to remind from which dataset the data contain. We also want to add a comment duplicate, which will link the comment 'id' to its matched comment. Naturally a comment id column would be added too. Finally the columns beer_id, brewery_id et user_id would be link to the one of the new id given in full_users, full_breweries and full_beers.

In [5]:
ratings_ratebeer['dataset'] = 'rb'
ratings_advocate['dataset'] = 'ad'

final_id_ratebeer = len(ratings_ratebeer) + 1
first_id_advocate = final_id_ratebeer
final_id_advocate = first_id_advocate + len(ratings_advocate)
ratings_ratebeer['id_rating'] = range(1, final_id_ratebeer) #For the moment call it id_rating, rename it later 
ratings_advocate['id_rating'] = range(first_id_advocate, final_id_advocate)

assert ratings_ratebeer.iloc[len(ratings_ratebeer)-1]['id_rating']!=ratings_advocate.iloc[0]['id_rating']

Give new brewery id

In [6]:
ratings_ratebeer = ratings_ratebeer.merge(full_breweries[['old_ratebeer_id', 'id']],how='left',left_on='brewery_id',right_on='old_ratebeer_id')
ratings_ratebeer = ratings_ratebeer.drop(columns='old_ratebeer_id')
ratings_ratebeer = ratings_ratebeer.rename(columns={'id':'id_brewery'})

# Convert the Series to a list and apply ast.literal_eval to each element if it's a string
result = [ast.literal_eval(x) if isinstance(x, str) else x for x in full_breweries['old_advocate_id'].tolist()]
full_breweries['old_advocate_id'] = pd.Series(result, index=full_breweries.index)
full_breweries = full_breweries.explode('old_advocate_id')

ratings_advocate = ratings_advocate.merge(full_breweries[['old_advocate_id', 'id']],how='left',left_on='brewery_id',right_on='old_advocate_id')
ratings_advocate = ratings_advocate.drop(columns='old_advocate_id')
ratings_advocate = ratings_advocate.rename(columns={'id':'id_brewery'})

Give new user id

In [7]:
ratings_advocate = ratings_advocate.merge(full_users[['old_user_id_advocate', 'id']],how='left',left_on='user_id',right_on='old_user_id_advocate')
ratings_advocate = ratings_advocate.drop(columns='old_user_id_advocate')
ratings_advocate = ratings_advocate.rename(columns={'id':'id_user'})

# Convert the Series to a list and apply ast.literal_eval to each element if it's a string
result = [ast.literal_eval(x) if isinstance(x, str) else x for x in full_users['old_user_id_ratebeer'].tolist()]
full_users['old_user_id_ratebeer'] = pd.Series(result, index=full_users.index)
full_users = full_users.explode('old_user_id_ratebeer')

ratings_ratebeer = ratings_ratebeer.merge(full_users[['old_user_id_ratebeer', 'id']],how='left',left_on='user_id',right_on='old_user_id_ratebeer')
ratings_ratebeer = ratings_ratebeer.drop(columns='old_user_id_ratebeer')
ratings_ratebeer = ratings_ratebeer.rename(columns={'id':'id_user'})

Get new beer id

In [ ]:
ratings_ratebeer = ratings_ratebeer.merge(full_beers[['old_beer_id_ratebeer', 'id']],how='left',left_on='beer_id',right_on='old_beer_id_ratebeer')
ratings_ratebeer = ratings_ratebeer.drop(columns='old_beer_id_ratebeer')
ratings_ratebeer = ratings_ratebeer.rename(columns={'id':'id_beer'})

# Convert the Series to a list and apply ast.literal_eval to each element if it's a string
result = [ast.literal_eval(x) if isinstance(x, str) else x for x in full_beers['old_beer_id_advocate'].tolist()]
full_beers['old_beer_id_advocate'] = pd.Series(result, index=full_beers.index)
full_beers = full_beers.explode('old_beer_id_advocate')

ratings_advocate = ratings_advocate.merge(full_beers[['old_beer_id_advocate', 'id']],how='left',left_on='beer_id',right_on='old_beer_id_advocate')
ratings_advocate = ratings_advocate.drop(columns='old_beer_id_advocate')
ratings_advocate = ratings_advocate.rename(columns={'id':'id_beer'})

Find the matched commentss

In [10]:
ratings_advocate_matched = copy.deepcopy(ratings_advocate)
ratings_ratebeer_matched = copy.deepcopy(ratings_ratebeer)

ratings_advocate_matched['matched'] = np.nan
ratings_ratebeer_matched['matched'] = np.nan

# Create a dictionary to map (user_id, beer_id) to id_rating for both datasets
advocate_dict = ratings_advocate_matched.set_index(['user_id', 'beer_id'])['id_rating'].to_dict()
ratebeer_dict = ratings_ratebeer_matched.set_index(['user_id', 'beer_id'])['id_rating'].to_dict()

for index, row in ratings_matched.iterrows():
    beer_advocate = row['beer_id']
    beer_ratebeer = row['beer_id.1']
    user_advocate = row['user_id']
    user_ratebeer = row['user_id.1']

    id_advocate = advocate_dict.get((user_advocate, beer_advocate))
    id_ratebeer = ratebeer_dict.get((user_ratebeer, beer_ratebeer))

    if id_advocate is not None and id_ratebeer is not None:
        ratings_advocate_matched.loc[ratings_advocate_matched['id_rating'] == id_advocate, 'matched'] = id_ratebeer
        ratings_ratebeer_matched.loc[ratings_ratebeer_matched['id_rating'] == id_ratebeer, 'matched'] = id_advocate

ratings_advocate_matched = ratings_advocate_matched.drop(columns=['beer_id', 'brewery_id', 'user_id'])
ratings_ratebeer_matched = ratings_ratebeer_matched.drop(columns=['beer_id', 'brewery_id', 'user_id'])

ratings_advocate_matched = ratings_advocate_matched.rename(columns={'id_rating': 'id'})
ratings_ratebeer_matched = ratings_ratebeer_matched.rename(columns={'id_rating': 'id'})


In [13]:
print(ratings_ratebeer_matched['id_brewery'].isna().sum())
print(ratings_ratebeer_matched['id_user'].isna().sum())
print(ratings_ratebeer_matched['id_beer'].isna().sum())
print(ratings_advocate_matched['id_brewery'].isna().sum())
print(ratings_advocate_matched['id_user'].isna().sum())
print(ratings_advocate_matched['id_beer'].isna().sum())
print("length of the dataset:",len(full_beers))


0
23
0
0
0
0
length of the dataset: 677264


Wait why is there no id_user for certain comments in ratebeer?

In [21]:
print(ratings_ratebeer[ratings_ratebeer['id_user'].isna()]['user_id'].unique())
print(full_users[full_users.old_user_id_ratebeer==474943  ])
print(full_users[full_users.old_user_id_ratebeer=='474943  '])

users_RATEBEER = pd.read_csv(os.path.join(RATEBEER_PATH, 'users.csv'))
print(users_RATEBEER[users_RATEBEER.user_id=='474943 '])
print(users_RATEBEER[users_RATEBEER.user_id=='474943 '])
del users_RATEBEER

[461238 362272 474943]
Empty DataFrame
Columns: [joined_advocate, old_user_id_advocate, user_name, joined_ratebeer, location, old_user_id_ratebeer, nbr_ratings_total, id]
Index: []
Empty DataFrame
Columns: [joined_advocate, old_user_id_advocate, user_name, joined_ratebeer, location, old_user_id_ratebeer, nbr_ratings_total, id]
Index: []
Empty DataFrame
Columns: [nbr_ratings, user_id, user_name, joined, location]
Index: []
Empty DataFrame
Columns: [nbr_ratings, user_id, user_name, joined, location]
Index: []


So the id doesn't exist in ratebeer...

In [ ]:
ratings_advocate
print(ratings_ratebeer['id_brewery'].isna().sum())
print(ratings_ratebeer['id_user'].isna().sum())
print(ratings_ratebeer['id_beer'].isna().sum())
print(ratings_advocate['id_brewery'].isna().sum())
print(ratings_advocate['id_user'].isna().sum())
print(ratings_advocate['id_beer'].isna().sum())

0
23
0
0
0
0


In [14]:
full_ratings = pd.concat([ratings_ratebeer_matched,ratings_advocate_matched], ignore_index=True)
full_ratings.sample(2)

,date,appearance,aroma,palate,taste,overall,rating,text,dataset,id,id_brewery,id_user,id_beer,matched
10979185,1440064800,4.25,4.25,4.25,4.0,4.25,4.15,NaN,ad,10977454,13398,72174.0,551564,NaN
12966988,1342346400,NaN,NaN,NaN,NaN,NaN,4.50,NaN,ad,12965257,6118,73813.0,33031,NaN


In [22]:
print(len(full_ratings))
print(len(ratings_advocate)+len(ratings_ratebeer))
print(len(full_ratings[~full_ratings['matched'].isna()]))
print(len(ratings_matched)*2)#Cause we have double the amount of comment as in match

15516838
15516838
43928
43928


In [18]:
full_ratings.to_csv(os.path.join(FULL_PATH,'ratings.csv'),index=False)

In [ ]:
test_rating = pd.read_csv(os.path.join(FULL_PATH,'ratings.csv'))
assert len(test_rating) == len(full_ratings), f"Expected {len(full_beers)}, but got {len(test_beer)}"
del test_rating